In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings 
from langchain.text_splitter import CharacterTextSplitter 
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate
from langchain import OpenAI, ConversationChain

from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Import PyPDF2
import PyPDF2
import numpy as np 

In [ ]:
import os 
# Load openai api key 
openai_api_key = os.environ.get('OPENAI_API_KEY')

# Load Pinecone API key 
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Load pinecode api env 
pinecone_api_env = os.environ.get('PINECONE_API_ENV')

In [ ]:
# Extract text from the PDF

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfFileReader(file)
        text = ''
        for page_num in range(reader.numPages):
            text += reader.getPage(page_num).extractText()
    return text

def get_all_pdf_files_in_directory(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.pdf')]

In [ ]:
import pinecone

# Initialize a Pinecone client

pinecone.init(api_key=pinecone_api_key, api_env=pinecone_api_env)
#help(pinecone.init)

In [ ]:
# Pick a name for the new index
index_name = 'semantic-text-search'

In [ ]:
import pandas as pd
import numpy as np
import time
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import sqlite3

pd.set_option('display.max_colwidth', 200)

In [ ]:
import os
import PyPDF2
import pinecone
from sentence_transformers import SentenceTransformer

In [ ]:
# Generate vector embeddings
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [ ]:
data_directory = "data/"
pdf_files = get_all_pdf_files_in_directory(data_directory)

In [ ]:
bnr_report_reader = UnstructuredPDFLoader("data/Annual_Report_2021_22_Web_English_Versio.pdf")
bnr_report_reader_data = bnr_report_reader.load()

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
# conversation = ConversationChain(llm=llm, verbose=True)

In [ ]:
conversation = ConversationChain(llm=llm, verbose=True)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(bnr_report_reader_data, OpenAIEmbeddings(openai_api_key=openai_api_key))
docs = faiss_index.similarity_search("What are the top economic challenges facing Rwanda?", k=2)

In [33]:
for doc in docs:
    print(docs[0].page_content)

1

NBR Annual Report | 2021 - 2022

2

NBR Annual Report | 2021 - 2022

INTRODUCTION

FOREWORD BY THE GOVERNOR

THE YEAR 2021/22 IN NUMBERS

DELIVERING ON OUR STRATEGY

EXECUTIVE SUMMARY

Q&A

CHAPTER ONE: NBR CORPORATE GOVERNANCE

ORGANIZATION STRUCTURE

BOARD OF DIRECTORS AND ITS COMMITTEES

BOARD COMMITTEE FUNCTIONS AND ACTIVITIES

MANAGEMENT COMMITTEE

POLICY COMMITTEES

NBR STRATEGIC MANAGEMENT

RISK GOVERNANCE FRAMEWORK

INTERNAL AUDIT FUNCTION

CHAPTER TWO: MONETARY POLICY

MONETARY POLICY FORMULATION AND ECONOMIC RESEARCH

GLOBAL ECONOMIC DEVELOPMENTS AND OUTLOOK

DOMESTIC ECONOMIC PERFORMANCE

EXCHANGE RATE DEVELOPMENTS

LIQUIDITY CONDITIONS AND INTEREST RATES DEVELOPMENTS

MONEY AND CREDIT

INFLATION DEVELOPMENTS

BONDS DEVELOPMENTS

CHALLENGES

CHAPTER THREE: FINANCIAL SYSTEM STABILITY

REGULATION AND LICENSING

MICRO PRUDENTIAL SUPERVISION

PAYMENT SYSTEM OVERSIGHT ACTIVITIES

MACRO PRUDENTIAL SUPERVISION

CREDIT REPORTING SYSTEM

DEPOSIT GUARANTEE FUND

STRUCTURE AND PERFO

In [ ]:
# Pick a name for the new index
simple_index_name = 'nationalbank-index'

In [ ]:
# Check whether the index with the same name already exists
if simple_index_name in pinecone.list_indexes():
    pinecone.delete_index(simple_index_name)

In [ ]:
# Create a new index
pinecone.create_index(name=simple_index_name, dimension=128)

In [ ]:
pinecone.list_indexes()

In [ ]:
pinecone.describe_index("pinecone-index")

In [ ]:
def extract_named_entities(text):
    import spacy
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return [(X.text, X.label_) for X in doc.ents]

In [ ]:
from pprint import pprint
# Define retriever
retriever = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Define index
index = pinecone.Index()

def search_pinecone(query):
    # extract named entities from the query
    ne = extract_named_entities([query])[0]
    # create embeddings for the query
    xq = retriever.encode(query).tolist()
    # query the pinecone index while applying named entity filter
    xc = index.query(xq, top_k=10, include_metadata=True, filter={"named_entities": {"$in": ne}})
    # extract article titles from the search result
    r = [x["metadata"]["title"] for x in xc["matches"]]
    return pprint({"Extracted Named Entities": ne, "Result": r})

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(openai_api_key=openai_api_key))
docs = faiss_index.similarity_search("What are the top economic challenges facing Rwanda?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)
print (f'Now you have {len(texts)} documents')

In [ ]:
import pinecone

pinecone.init(api_key=pinecone_api_key)

pinecone.create_index("national_bank_index", dimension=1024)